In [90]:
'''Trains a simple deep NN on the MNIST dataset.
Gets to 98.40% test accuracy after 20 epochs
(there is *a lot* of margin for parameter tuning).
2 seconds per epoch on a K520 GPU.
'''

from __future__ import print_function

import keras
from sklearn.metrics import confusion_matrix
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
import numpy as np

def transform(twoD):
    length = twoD.shape[0]
    oneD = np.zeros(length)
    for i in range(10000):
        max = -1
        val = 0
        for j in range(10):
            if twoD[i][j] > val:
                max = j 
                val = twoD[i][j]
        oneD[i] = max
    return oneD

def normalize(m):
    for i in range(m.shape[0]):
        sum = 0
        for j in range(m.shape[1]):
            sum = sum + m[i][j]
        for j in range(m.shape[1]): 
            m[i][j] = m[i][j]/sum
    return m

batch_size = 128
num_classes = 10
epochs = 20

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
# origin version
# model.add(Dense(512, activation='relu', input_shape=(784,)))
# model.add(Dropout(0.2))
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(10, activation='softmax'))

# network for Q1
# model.add(Dense(10, activation='softmax', input_shape=(784,)))

# final version
model.add(Dense(256, activation='relu', input_shape=(784,)))
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.summary()

model.compile(loss='mean_squared_error',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size, epochs=epochs,
                    verbose=1, validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
result = model.predict(x_test, batch_size=32, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

60000 train samples
10000 test samples
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 256)               200960    
_________________________________________________________________
dense_18 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_19 (Dense)             (None, 10)                2570      
Total params: 269,322.0
Trainable params: 269,322
Non-trainable params: 0.0
_________________________________________________________________
Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 4s - loss: 0.0121 - acc: 0.9196 - val_loss: 0.0074 - val_acc: 0.9512

In [91]:
# transform 2-D prediction into 1-D array
y_pred = transform(result)
y_test = transform(y_test)

# create confusion matrix based on two ground_truth and prediction
confusionM = confusion_matrix(y_test, y_pred)

# cast int8 np array into float format
normal = np.float64(confusionM)

# set output np precision
np.set_printoptions(precision=4)

final = normalize(normal)
print(final)

[[  9.9082e-01   0.0000e+00   0.0000e+00   1.0204e-03   2.0408e-03
    0.0000e+00   2.0408e-03   1.0204e-03   3.0612e-03   0.0000e+00]
 [  2.6432e-03   9.9119e-01   8.8106e-04   0.0000e+00   0.0000e+00
    1.7621e-03   1.7621e-03   0.0000e+00   1.7621e-03   0.0000e+00]
 [  4.8450e-03   9.6899e-04   9.8062e-01   2.9070e-03   1.9380e-03
    0.0000e+00   0.0000e+00   6.7829e-03   1.9380e-03   0.0000e+00]
 [  0.0000e+00   0.0000e+00   4.9505e-03   9.8218e-01   0.0000e+00
    3.9604e-03   0.0000e+00   3.9604e-03   1.9802e-03   2.9703e-03]
 [  1.0183e-03   0.0000e+00   2.0367e-03   0.0000e+00   9.8065e-01
    2.0367e-03   2.0367e-03   1.0183e-03   0.0000e+00   1.1202e-02]
 [  2.2422e-03   0.0000e+00   0.0000e+00   2.2422e-03   1.1211e-03
    9.8767e-01   3.3632e-03   1.1211e-03   2.2422e-03   0.0000e+00]
 [  4.1754e-03   2.0877e-03   1.0438e-03   1.0438e-03   4.1754e-03
    2.0877e-03   9.8434e-01   1.0438e-03   0.0000e+00   0.0000e+00]
 [  9.7276e-04   2.9183e-03   4.8638e-03   9.7276e-04  